##Abstractive

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [3]:
# initialize the pretrained model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
#Generate Summary
def abstractive_summary(text):
  ## preprocess the input text
  preprocessed_text = text.strip().replace('\n','')
  t5_input_text = 'summarize: ' + preprocessed_text
  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

  summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary

##Extractive

In [51]:
# import the required libraries
import nltk
nltk.download('punkt') # punkt tokenizer for sentence tokenization
nltk.download('stopwords') # list of stop words, such as 'a', 'an', 'the', 'in', etc, which would be dropped
from collections import Counter # Imports the Counter class from the collections module, used for counting the frequency of words in a text.
from nltk.corpus import stopwords # Imports the stop words list from the NLTK corpus
# corpus is a large collection of text or speech data used for statistical analysis

from nltk.tokenize import sent_tokenize, word_tokenize # Imports the sentence tokenizer and word tokenizer from the NLTK tokenizer module.
# Sentence tokenizer is for splitting text into sentences
# word tokenizer is for splitting sentences into words

# this function would take 2 inputs, one being the text, and the other being the summary which would contain the number of lines
def extractive_summary(text):
  # Tokenize the text into individual sentences
  sentences = sent_tokenize(text)

  # Tokenize each sentence into individual words and remove stopwords
  stop_words = set(stopwords.words('english'))
  # the following line would tokenize each sentence from sentences into individual words using the word_tokenize function of nltk.tokenize module
  # Then removes any stop words and non-alphanumeric characters from the resulting list of words and converts them all to lowercase
  words = [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words and word.isalnum()]

  # Compute the frequency of each word
  word_freq = Counter(words)

  # Compute the score for each sentence based on the frequency of its words
  # After this block of code is executed, sentence_scores will contain the scores of each sentence in the given text,
  # where each score is a sum of the frequency counts of its constituent words

  # empty dictionary to store the scores for each sentence
  sentence_scores = {}

  for sentence in sentences:
    sentence_words = [word.lower() for word in word_tokenize(sentence) if word.lower() not in stop_words and word.isalnum()]
    sentence_score = sum([word_freq[word] for word in sentence_words])
  if len(sentence_words) < 20:
    sentence_scores[sentence] = sentence_score

  # checks if the length of the sentence_words list is less than 20 (parameter can be adjusted based on the desired length of summary sentences)
  # If condition -> true, score of the current sentence is added to the sentence_scores dictionary with the sentence itself as the key
  # This is to filter out very short sentences that may not provide meaningful information for summary generation

  # Select the top n sentences with the highest scores
  summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:1]
  summary = ' '.join(summary_sentences)

  summary_sentences2 = summary.split('. ')
  formatted_summary = '.\n'.join(summary_sentences2)

  return formatted_summary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
